# Plot literature review of conductivities and permittivities

The original Gabriel model (https://doi.org/10.1088/0031-9155/41/11/003) is compared against an algorithmic correction by Zimmermann and van Rienen (https://doi.org/10.1016/j.bioelechem.2021.107773).
For comparison, the data by Wagner et al. is shown (taken from Supp. Fig. 2 of https://doi.org/10.1016/j.neuroimage.2013.06.079).

The parameter values of the original Gabriel model are available online: http://niremf.ifac.cnr.it/docs/DIELECTRIC/AppendixC.html

This website is also archived here: https://web.archive.org/web/20240718162705/http://niremf.ifac.cnr.it/docs/DIELECTRIC/AppendixC.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import epsilon_0 as e0

## 4 Cole-Cole model

This is an implementation of the 4 Cole-Cole model of the complex permittivity suggested by Gabriel et al.
The source code is mostly taken from ImpedanceFitter (https://github.com/j-zimmermann/impedancefitter).
For comparison, also the Havriliak-Negami model that was used to fit the data by Zimmermann and van Rienen is provided.

In a helper function, the complex permittivity can be converted into relative permittivity and conductivity.

In [ ]:
def cole_cole_4_model(
    omega,
    epsinf,
    deps1,
    deps2,
    deps3,
    deps4,
    tau1,
    tau2,
    tau3,
    tau4,
    a1,
    a2,
    a3,
    a4,
    sigma,
):
    r"""Standard 4-Cole-Cole impedance model.


    Parameters
    ----------
    omega: :class:`numpy.ndarray`, double
        list of frequencies
    epsinf: double
        value for :math:`\varepsilon_\infty`
    deps1: double
        value for :math:`\Delta\varepsilon_1`
    deps2: double
        value for :math:`\Delta\varepsilon_2`
    deps3: double
        value for :math:`\Delta\varepsilon_3`
    deps4: double
        value for :math:`\Delta\varepsilon_4`
    tau1: double
        value for :math:`\tau_1`, in ps
    tau2: double
        value for :math:`\tau_2`, in ns
    tau3: double
        value for :math:`\tau_3`, in us
    tau4: double
        value for :math:`\tau_4`, in ms
    a1: double
        value for :math:`1 - \alpha_1 = a`
    a2: double
        value for :math:`1 - \alpha_2 = a`
    a3: double
        value for :math:`1 - \alpha_3 = a`
    a4: double
        value for :math:`1 - \alpha_4 = a`
    sigma: double
        conductivity value

    Returns
    -------
    :class:`numpy.ndarray`, complex
        Complex permittivity array

    Notes
    -----
    The original model has been described in [Gabriel1996]_.

    References
    ----------
    .. [Gabriel1996] Gabriel, S., Lau, R. W., & Gabriel, C. (1996).
                    The dielectric properties of biological tissues:
                    III. Parametric models for the dielectric spectrum of tissues.
                    Physics in Medicine and Biology, 41(11), 2271–2293.
                    https://doi.org/10.1088/0031-9155/41/11/003
    """
    tau1 *= 1e-12
    tau2 *= 1e-9
    tau3 *= 1e-6
    tau4 *= 1e-3
    epsc = epsinf - 1j * sigma / (omega * e0)

    epsc += deps1 / (1.0 + np.power((1j * omega * tau1), a1))
    epsc += deps2 / (1.0 + np.power((1j * omega * tau2), a2))
    epsc += deps3 / (1.0 + np.power((1j * omega * tau3), a3))
    epsc += deps4 / (1.0 + np.power((1j * omega * tau4), a4))
    return epsc

def havriliak_negami(omega, epsinf, deps, tau, a, beta, sigma):
    r"""Havriliak-Negami relaxation.

    Parameters
    ----------
    omega: :class:`numpy.ndarray`, double
        list of frequencies
    epsinf: double
        value for :math:`\varepsilon_\infty`
    deps: double
        value for :math:`\Delta\varepsilon`
    tau: double
        value for :math:`\tau`, in ns
    sigma: double
        value for :math:`\sigma_\mathrm{dc}`
    a: double
        value for :math:`1 - \alpha = a`
    beta: double
        value for :math:`\beta`

    Returns
    -------
    :class:`numpy.ndarray`, complex
        Impedance array


    Notes
    -----
    .. warning::

        The unit capacitance is in pF!
        The time constant tau is in ns!

    Equations for calculations:

    .. math::

        \varepsilon^\ast = \varepsilon_\infty +
                           \frac{\Delta\varepsilon}
                           {\left(1 + (j \omega \tau)^{a}\right)^\beta} -
                           \frac{j\sigma_{\mathrm{DC}}}{\omega \varepsilon_0}
                           \enspace ,

    .. math::

        Z = \frac{1}{j\varepsilon^\ast \omega c_\mathrm{0}}

    """
    deps *= 1e3
    tau *= 1e-6
    
    epsc = (
        epsinf
        + deps / np.power(1.0 + np.power(1j * omega * tau, a), beta)
        - 1j * sigma / (omega * e0)
    )

    return epsc

def return_diel_properties(omega, epsc):
    r"""Return relative permittivity and conductivity from complex permittivity.

    Notes
    -----

    The relative permittivity is the real part of :math:`\varepsilon_\mathrm{r}^\ast`
    and the conductivity is the negative imaginary part times the frequency and the
    vacuum permittivity (see, e.g., [Grant1958]_.

    Parameters
    ----------
    omega: :class:`numpy.ndarray`, double
        frequency array
    epsc: :class:`numpy.ndarray`, complex
        complex permittivity array

    Returns
    -------
    eps_r: :class:`numpy.ndarray`, double
        relative permittivity
    conductivity: :class:`numpy.ndarray`, double
        conductivity in S/m

    References
    ----------
    .. [Grant1958] Grant, F. A. (1958).
                   Use of complex conductivity in the
                   representation of dielectric phenomena.
                   Journal of Applied Physics, 29(1), 76–80.
                   https://doi.org/10.1063/1.1722949
    """
    eps_r = epsc.real
    conductivity = -epsc.imag * e0 * omega
    return eps_r, conductivity

# Data by Wagner et al.

In the following cell, the Wagner data is shared.
It is organised as : 
Frequency (Hz), Conductivity (S/m), Relative Permittivity


In [ ]:
def get_wagner_data():
    wagner_data = np.array(
    [[10.0000, 0.32080, 31630000.0],
    [11.2200, 0.32170, 29940000.0],
    [12.5890, 0.32860, 28450000.0],
    [14.1250, 0.32250, 27500000.0],
    [15.8480, 0.32240, 26450000.0],
    [17.7820, 0.32560, 25120000.0],
    [19.9520, 0.32720, 23800000.0],
    [22.3870, 0.32770, 22400000.0],
    [25.1180, 0.33360, 21290000.0],
    [28.1830, 0.33370, 21280000.0],
    [31.6220, 0.33650, 21270000.0],
    [35.4810, 0.33770, 21500000.0],
    [39.8100, 0.33850, 21730000.0],
    [44.6680, 0.34310, 21230000.0],
    [50.1180, 0.35760, 20930000.0],
    [56.2340, 0.35860, 19880000.0],
    [63.0950, 0.35910, 19200000.0],
    [70.7940, 0.33610, 18490000.0],
    [79.4320, 0.34830, 17590000.0],
    [89.1250, 0.33510, 16240000.0],
    [100.0000, 0.33360, 15280000.0],
    [112.2010, 0.33470, 14650000.0],
    [125.8920, 0.33930, 13870000.0],
    [141.2530, 0.34270, 13090000.0],
    [158.4890, 0.34320, 12260000.0],
    [177.8270, 0.34580, 11260000.0],
    [199.5260, 0.34460, 10560000.0],
    [223.8720, 0.34690, 9720000.0],
    [251.1880, 0.35180, 9293000.0],
    [281.8380, 0.35320, 8774000.0],
    [316.2270, 0.35410, 8316000.0],
    [354.8130, 0.35710, 7746000.0],
    [398.1070, 0.35730, 7187000.0],
    [446.6830, 0.35850, 6743000.0],
    [501.1870, 0.35760, 6177000.0],
    [562.3410, 0.36010, 5790000.0],
    [630.9570, 0.36360, 5394000.0],
    [707.9450, 0.37360, 5110000.0],
    [794.3280, 0.38790, 4796000.0],
    [891.2500, 0.39640, 4408000.0],
    [1000.0000, 0.40260, 4091000.0],
    [1122.0180, 0.40850, 3724000.0],
    [1258.9250, 0.41040, 3433000.0],
    [1412.5370, 0.41440, 3104000.0],
    [1584.8930, 0.42740, 2951000.0],
    [1778.2790, 0.43900, 2700000.0],
    [1995.2620, 0.46540, 2592000.0],
    [2238.7210, 0.48410, 2398000.0],
    [2511.8860, 0.51010, 2128000.0],
    [2818.3820, 0.54550, 1939000.0],
    [3162.2770, 0.54860, 1718000.0],
    [3548.1330, 0.56360, 1593000.0],
    [3981.0710, 0.57130, 1419000.0],
    [4466.8350, 0.59510, 1317000.0],
    [5011.8720, 0.61280, 1192000.0],
    [5623.4130, 0.63200, 1064000.0],
    [6309.5730, 0.64270, 944100.0],
    [7079.4570, 0.64220, 819700.0],
    [7943.2820, 0.65870, 732100.0],
    [8912.5090, 0.63270, 661400.0],
    [10000.0000, 0.63350, 607100.0],
    [11220.1800, 0.62270, 464900.0],
    [12589.2500, 0.62940, 405400.0],
    [14125.3700, 0.63830, 355800.0],
    [15848.9300, 0.65710, 312100.0],
    [17782.7900, 0.64870, 263400.0],
    [19952.6200, 0.66910, 232800.0],
    [22387.2100, 0.66730, 197800.0],
    [25118.8600, 0.69100, 176700.0],
    [28183.8200, 0.70110, 152100.0],
    [31622.7700, 0.73000, 136900.0],
    [35481.3300, 0.74250, 117700.0],
    [39810.7100, 0.76340, 104800.0],
    [44668.3500, 0.76660, 88760.0],
    [50118.7200, 0.76630, 77490.0]])
    f = wagner_data[:, 0]
    sigma = wagner_data[:, 1]
    permittivity = wagner_data[:, 2]
    return f, sigma, permittivity 

# Plots of the dielectric properties of brain tissue

The dielectric properties are plotted between 1 Hz and 10 kHz.
Grey and white matter data are plotted.

The Zimmermann / van Rienen model removes the fourth dispersion and adjusts the Gabriel model.
Otherwise, all parameter values are the same as in the Gabriel model.
In addition, a Havriliak-Negami was explored and fitted against the raw Gabriel data.
The Wagner data is measurement data and was not described by a model.

In [ ]:
frequencies = np.logspace(1, 4)
omega = 2.0 * np.pi * frequencies

In [ ]:
# Gray matter data
wagner_frequencies, wagner_conductivities, wagner_permittivities = get_wagner_data()
review_dict = {}
review_dict['Gabriel et al. (1996)'] = {"epsinf": 4.0, 
                                  "deps1": 45, 
                                  "tau1": 7.958,
                                  "a1": 0.9,
                                  "deps2": 400, 
                                  "tau2": 15.915,
                                  "a2": 0.85,
                                  "sigma": 0.02,
                                  "deps3": 200000.0, 
                                  "tau3": 106.103, 
                                  "a3": 0.78,
                                  "deps4": 45000000.0,
                                  "tau4": 5.305,
                                  "a4": 1.0
                                  }

review_dict["Zimmermann, van Rienen (2021)"] = review_dict["Gabriel et al. (1996)"].copy()
review_dict["Zimmermann, van Rienen (2021)"]["deps4"] = 0
review_dict["Zimmermann, van Rienen (2021)"]["sigma"] = 0.10697690700220125

fig = plt.figure(figsize=[6, 6])
fig.subplots_adjust(bottom=0.17, left=0.1, right=0.7, hspace=0.45)
ax1 = fig.add_subplot(311,
                      ylabel='S/m',
                     ylim=[0.0, 0.7]
                     )
ax1.grid(True)
ax2= fig.add_subplot(312, 
                      sharex=ax1,
                     ylim=[0, 4e7]
                     )
ax2.grid(True)
ax3 = fig.add_subplot(313, 
                      sharex=ax1,
                    xlabel='$f$ (Hz)',
                      ylim=[0, 1]
                     )
ax3.grid(True)

keys = review_dict.keys()

for key in keys:
    parameters = review_dict[key]
    factor = 1
    if "Zimmermann" in key:
        factor = 100
        ax2.text(1e3, 1.5e7, "100x", color="tab:orange")
    if "HN" in key:
        epsc = havriliak_negami(omega, **parameters)
    else:
        epsc = cole_cole_4_model(omega, **parameters)
    eps_r, conductivity = return_diel_properties(omega, epsc)
    ax1.semilogx(frequencies, conductivity, label=key, lw=2)
    ax2.semilogx(frequencies, factor * eps_r, label=key, lw=2)
    ax3.semilogx(frequencies, omega* e0* eps_r / conductivity, label=key, lw=2)

# plot Wagner data
wagner_label = "Wagner et al. (2014)"
ax1.semilogx(wagner_frequencies, wagner_conductivities, lw=2, label=wagner_label)
ax2.semilogx(wagner_frequencies, wagner_permittivities, lw=2, label=wagner_label)
ax3.semilogx(wagner_frequencies,
             wagner_frequencies * 2.0 * np.pi * e0 * wagner_permittivities / wagner_conductivities,
             lw=2, label=wagner_label)

ax1.set_title(r"$\sigma_\mathrm{rt}$")
ax2.set_title(r"$\varepsilon_\mathrm{rt}$")
ax2.legend(fontsize=8)
ax3.set_title(r"$2\pi\varepsilon_\mathrm{rt}/\sigma_\mathrm{rt}$")

plt.xlim(None, 1e4)

plt.savefig("gray_matter_review.pdf")
plt.show()

In [ ]:
# Gray matter data
wagner_frequencies, wagner_conductivities, wagner_permittivities = get_wagner_data()
review_dict = {}
review_dict['Gabriel et al. (1996)'] = {"epsinf": 4.0, 
                                  "deps1": 45, 
                                  "tau1": 7.958,
                                  "a1": 0.9,
                                  "deps2": 400, 
                                  "tau2": 15.915,
                                  "a2": 0.85,
                                  "sigma": 0.02,
                                  "deps3": 200000.0, 
                                  "tau3": 106.103, 
                                  "a3": 0.78,
                                  "deps4": 45000000.0,
                                  "tau4": 5.305,
                                  "a4": 1.0
                                  }

review_dict["Zimmermann, van Rienen (2021)"] = review_dict["Gabriel et al. (1996)"].copy()
review_dict["Zimmermann, van Rienen (2021)"]["deps4"] = 0
review_dict["Zimmermann, van Rienen (2021) HN"] = {"epsinf": 11.923084587912314, 
                                                   "deps": 95.55143380801692, 
                                                   "tau": 170.90781106059063, 
                                                   "a": 1.0, 
                                                   "beta": 0.586562773136095,
                                                   "sigma": 0.10697690700220125}
review_dict["Zimmermann, van Rienen (2021)"]["sigma"] = review_dict["Zimmermann, van Rienen (2021) HN"]["sigma"]

fig = plt.figure(figsize=[6, 6])
fig.subplots_adjust(bottom=0.17, left=0.1, right=0.7, hspace=0.45)
ax1 = fig.add_subplot(311,
                      ylabel='S/m',
                     ylim=[0.0, 0.7]
                     )
ax1.grid(True)
ax2= fig.add_subplot(312, 
                      sharex=ax1,
                     ylim=[0, 4e7]
                     )
ax2.grid(True)
ax3 = fig.add_subplot(313, 
                      sharex=ax1,
                    xlabel='$f$ (Hz)',
                      ylim=[0, 1]
                     )
ax3.grid(True)

keys = review_dict.keys()

for key in keys:
    parameters = review_dict[key]
    factor = 1
    if "Zimmermann" in key:
        factor = 100
        ax2.text(1e3, 1.5e7, "100x", color="tab:orange")
    if "HN" in key:
        epsc = havriliak_negami(omega, **parameters)
    else:
        epsc = cole_cole_4_model(omega, **parameters)
    eps_r, conductivity = return_diel_properties(omega, epsc)
    ax1.semilogx(frequencies, conductivity, label=key, lw=2)
    ax2.semilogx(frequencies, factor * eps_r, label=key, lw=2)
    ax3.semilogx(frequencies, omega* e0* eps_r / conductivity, label=key, lw=2)

# plot Wagner data
wagner_label = "Wagner et al. (2014)"
ax1.semilogx(wagner_frequencies, wagner_conductivities, lw=2, label=wagner_label)
ax2.semilogx(wagner_frequencies, wagner_permittivities, lw=2, label=wagner_label)
ax3.semilogx(wagner_frequencies,
             wagner_frequencies * 2.0 * np.pi * e0 * wagner_permittivities / wagner_conductivities,
             lw=2, label=wagner_label)

ax1.set_title(r"$\sigma_\mathrm{rt}$")
ax2.set_title(r"$\varepsilon_\mathrm{rt}$")
ax2.legend(fontsize=8)
ax3.set_title(r"$2\pi\varepsilon_\mathrm{rt}/\sigma_\mathrm{rt}$")

plt.xlim(None, 1e4)
plt.show()

## Plot model over entire frequency range of Wagner data

In the previous cell, the range was limited to frequencies up to 10 kHz.
Here, we evaluate the two analytical models for the entire frequency range
of the Wagner data.

In [ ]:
# Gray matter data
wagner_frequencies, wagner_conductivities, wagner_permittivities = get_wagner_data()
wagner_omega = 2.0 * np.pi * wagner_frequencies

fig = plt.figure(figsize=[6, 6])
fig.subplots_adjust(bottom=0.17, left=0.1, right=0.7, hspace=0.45)
ax1 = fig.add_subplot(311,
                      ylabel='S/m',
                     ylim=[0.0, 0.8]
                     )
ax1.grid(True)
ax2= fig.add_subplot(312, 
                      sharex=ax1,
                     ylim=[0, 4e7]
                     )
ax2.grid(True)
ax3 = fig.add_subplot(313, 
                      sharex=ax1,
                    xlabel='$f$ (Hz)',
                      ylim=[0, 1]
                     )
ax3.grid(True)

keys = review_dict.keys()

for key in keys:
    parameters = review_dict[key]
    factor = 1
    if "Zimmermann" in key:
        factor = 100
        ax2.text(1e3, 1.5e7, "100x", color="tab:orange")

    if "HN" in key:
        epsc = havriliak_negami(wagner_omega, **parameters)
    else:
        epsc = cole_cole_4_model(wagner_omega, **parameters)
    eps_r, conductivity = return_diel_properties(wagner_omega, epsc)
    ax1.semilogx(wagner_frequencies, conductivity, label=key, lw=2)
    ax2.semilogx(wagner_frequencies, factor * eps_r, label=key, lw=2)
    ax3.semilogx(wagner_frequencies, wagner_omega * e0 * eps_r / conductivity, label=key, lw=2)

# plot Wagner data
wagner_label = "Wagner et al. (2014)"
ax1.semilogx(wagner_frequencies, wagner_conductivities, lw=2, label=wagner_label)
ax2.semilogx(wagner_frequencies, wagner_permittivities, lw=2, label=wagner_label)
ax3.semilogx(wagner_frequencies,
             wagner_omega * e0 * wagner_permittivities / wagner_conductivities,
             lw=2, label=wagner_label)

ax1.set_title(r"$\sigma_\mathrm{rt}$")
ax2.set_title(r"$\varepsilon_\mathrm{rt}$")
ax2.legend(fontsize=8)
ax3.set_title(r"$2\pi\varepsilon_\mathrm{rt}/\sigma_\mathrm{rt}$")

plt.savefig("gray_matter_review_wagner.pdf")

In [ ]:
review_dict = {}
# White matter data
review_dict['Gabriel et al. (1996)'] = {"epsinf": 4.0, 
                                 "deps1": 32,
                                 "tau1": 7.958,
                                 "a1": 0.9,
                                 "deps2": 100,
                                 "tau2": 7.958,
                                 "a2": 0.9,
                                 "sigma": 0.02,
                                 "deps3": 40000.0,
                                 "tau3": 53.052,
                                 "a3": 0.7,
                                 "deps4": 35000000.0,
                                 "tau4": 7.958,
                                 "a4": 0.98}

review_dict["Zimmermann, van Rienen (2021)"] = review_dict["Gabriel et al. (1996)"].copy()
review_dict["Zimmermann, van Rienen (2021)"]["deps4"] = 0
review_dict["Zimmermann, van Rienen (2021) HN"] = {"epsinf": 7.772254985995842,
                                                   "deps": 49.02948959484944,
                                                   "tau": 159.76950536646012,
                                                   "a": 0.9849021030831951,
                                                   "beta": 0.5788078503096981,
                                                   "sigma": 0.061080374691163}
review_dict["Zimmermann, van Rienen (2021)"]["sigma"] = 0.061080374691163

fig = plt.figure(figsize=[6, 6])
fig.subplots_adjust(bottom=0.17, left=0.1, right=0.7, hspace=0.45)
ax1 = fig.add_subplot(311,
                      ylabel='S/m',
                     ylim=[0.0, 0.3]
                     )
ax1.grid(True)
ax2= fig.add_subplot(312, 
                      sharex=ax1,
                     ylim=[0, 3e7]
                     )
ax2.grid(True)
ax3 = fig.add_subplot(313, 
                      sharex=ax1,
                    xlabel='$f$ (Hz)',
                      ylim=[0, 1]
                     )
ax3.grid(True)

keys = review_dict.keys()

for key in keys:
    parameters = review_dict[key]
    factor = 1
    if "Zimmermann" in key:
        factor = 100
        ax2.text(1e3, 0.5e7, "100x", color="tab:orange")
    if "HN" in key:
        epsc = havriliak_negami(omega, **parameters)
    else:
        epsc = cole_cole_4_model(omega, **parameters)
    eps_r, conductivity = return_diel_properties(omega, epsc)
    ax1.semilogx(frequencies, conductivity, label=key, lw=2)
    ax2.semilogx(frequencies, factor * eps_r, label=key, lw=2)
    ax3.semilogx(frequencies, omega* e0* eps_r / conductivity, label=key, lw=2)


ax1.set_title(r"$\sigma_\mathrm{rt}$")
ax2.set_title(r"$\varepsilon_\mathrm{rt}$")
ax2.legend()
ax3.set_title(r"$2\pi\varepsilon_\mathrm{rt}/\sigma_\mathrm{rt}$")

plt.savefig('white_matter_review.pdf')